# Chapter VIII Cogeneration System / コージェネレーション設備

## 1. Introduction

This calculation depends on the specification of "Chapter VIII Cogeneration System" in BRI

## 3. Functions

### 3.1 Gas Consumption and Kerosene Consumption / ガス消費量及び灯油消費量

$$
E_{CG} = E_{F,PU} + E_{F,BB,DHW} + E_{F,BB,HWH}
$$

Where  
$E_{CG}$ is the daily primary energy consumption of cogeneration system / 1日当たりのコージェネレーション設備の一次エネルギー消費量 ( MJ/d );  
$E_{F,PU}$ is the daily fuel consumption of power generation unit / 1日当たりの発電ユニットの燃料消費量 ( MJ/d );  
$E_{F,BB,DHW}$ is the daily fuel consumption of backup boiler during hot water supply / 1日当たりの給湯時のバックアップボイラーの燃料消費量 ( MJ/d );  
$E_{F,BB,HWH}$ is the daily fuel consumption of backup boiler during hot water heating / 1日当たりの温水暖房時のバックアップボイラーの燃料消費量 ( MJ/d ).  

In [1]:
def get_E_CG( E_F_PU, E_F_BB_DHW, E_F_BB_HWH ):
    return E_F_PU + E_F_BB_DHW + E_F_BB_HWH

### 3.2 Amount of Power Generation / 発電量

i) When there is no use of exhaust heat to hot water heating:

$$
E_{E,CG,gen} = E_{E,gen,PU} - E_{E,BB,aux,ba2} - E_{E,BB,aux,HWH} - E_{E,TU,aux}
$$


ii) When exhaust heat is used for hot water heating:

$$
E_{E,CG,gen} = E_{E,gen,PU} - E_{E,BB,aux,ba2} - E_{E,TU,aux}
$$

Whether or not exhaust heat is used for hot water heating is in accordance with Appendix A.

Where  
$E_{E,CG,gen}$ is the hourly power generation by cogeneration system / 1時間当たりのコージェネレーション設備による発電量 ( kWh/h );  
$E_{E,gen,PU}$ is the hourly power generation of power generation unit / 1時間当たりの発電ユニットの発電量 ( kWh/h );  
$E_{E,BB,aux,ba2}$ is the hourly auxiliary machine power consumption of backup boiler during bathtub additional heating / 1時間当たりの浴槽追焚時のバックアップボイラーの補機消費電力量 ( kWh/h );  
$E_{E,BB,aux,HWH}$ is the hourly auxiliary machine power consumption of backup boiler during hot water heating / 1時間当たりの温水暖房時のバックアップボイラーの補機消費電力量 ( kWh/h );  
$E_{E,TU,aux}$ is the hourly auxiliary machine power consumption of the tank unit / 1時間当たりのタンクユニットの補機消費電力量 ( kWh/h ).  

In [3]:
def get_E_E_CG_gen( _E_E_gen_PU, _E_E_TU_aux ):
    return _E_E_gen_PU - _E_E_TU_aux



#### Example

In [6]:
get_E_E_CG_gen(10,8.5)

1.5

### 3.3 Fuel Consumption of Backup Boiler of Hot Water Supply / 給湯のバックアップボイラーの燃料消費量

#### 1) Fuel Consumption by Application / 用途別の燃料消費量

$$
E_{F,BB,DHW} = E_{F,BB,k} + E_{F,BB,s} + E_{F,BB,w} + E_{F,BB,b1} + E_{F,BB,b2} + E_{F,BB,ba1} + E_{F,BB,ba2}
$$

Where  
$E_{F,BB,DHW}$ is the daily fuel consumption of backup boiler during hot water supply / 1日当たりの給湯時のバックアップボイラーの燃料消費量 ( MJ/d );  
$E_{F,BB,k}$ is the daily fuel consumption of backup boiler by the kitchen / 1日当たりの台所水栓におけるバックアップボイラーの燃料消費量 ( MJ/d );  
$E_{F,BB,s}$ is the daily fuel consumption of backup boiler by the shower / 1日当たりの浴室シャワー水栓におけるバックアップボイラーの燃料消費量 ( MJ/d );  
$E_{F,BB,w}$ is the daily fuel consumption of backup boiler by the washroom / 1日当たりの洗面水栓におけるバックアップボイラーの燃料消費量 ( MJ/d );  
$E_{F,BB,b1}$ is the daily fuel consumption of backup boiler by filling the bathtub manually / 1日当たりの浴槽水栓湯はりにおけるバックアップボイラーの燃料消費量 ( MJ/d );  
$E_{F,BB,b2}$ is the daily fuel consumption of backup boiler by filling the bathtub automattically / 1日当たりの浴槽自動湯はりにおけるバックアップボイラーの燃料消費量 ( MJ/d );  
$E_{F,BB,ba1}$ is the daily fuel consumption of backup boiler by addition of hot water to bathtub / 1日当たりの浴槽水栓さし湯におけるバックアップボイラーの燃料消費量 ( MJ/d );  
$E_{F,BB,ba2}$ is the daily fuel consumption of backup boiler by rewarming the bathtub / 1日当たりの浴槽追焚におけるバックアップボイラーの燃料消費量 ( MJ/d ).  

In [10]:
def get_E_F_BB_DHW( E_F_BB_k, E_F_BB_s, E_F_BB_w, E_F_BB_b1, E_F_BB_b2, E_F_BB_ba1, E_F_BB_ba2 ):
    return E_F_BB_k + E_F_BB_s + E_F_BB_w + E_F_BB_b1 + E_F_BB_b2 + E_F_BB_ba1 + E_F_BB_ba2

$$
E_{F,BB,k} = L_{BB,k} \div e_{BB,k}
$$

$$
E_{F,BB,s} = L_{BB,s} \div e_{BB,s}
$$

$$
E_{F,BB,w} = L_{BB,w} \div e_{BB,w}
$$

$$
E_{F,BB,b1} = L_{BB,b1} \div e_{BB,b1}
$$

$$
E_{F,BB,b2} = L_{BB,b2} \div e_{BB,b2}
$$

$$
E_{F,BB,ba1} = L_{BB,ba1} \div e_{BB,ba1}
$$

$$
E_{F,BB,ba2} = L_{BB,ba2} \div e_{BB,ba2}
$$

Where  
$L_{BB,k}$ is the daily hot water heat load shared by backup boiler by the kitchen / 1日当たりの台所水栓におけるバックアップボイラーが分担する給湯熱負荷 ( MJ/d );  
$L_{BB,s}$ is the daily hot water heat load shared by backup boiler by the shower / 1日当たりの浴室シャワー水栓におけるバックアップボイラーが分担する給湯熱負荷 ( MJ/d );  
$L_{BB,w}$ is the daily hot water heat load shared by backup boiler by the washroom / 1日当たりの洗面水栓におけるバックアップボイラーが分担する給湯熱負荷 ( MJ/d );  
$L_{BB,b1}$ is the daily hot water heat load shared by backup boiler by filling the bathtub manually / 1日当たりの浴槽水栓湯はりにおけるバックアップボイラーが分担する給湯熱負荷 ( MJ/d );  
$L_{BB,b2}$ is the daily hot water heat load shared by backup boiler by filling the bathtub automattically / 1日当たりの浴槽自動湯はりにおけるバックアップボイラーが分担する給湯熱負荷 ( MJ/d );  
$L_{BB,ba1}$ is the daily hot water heat load shared by backup boiler by addition of hot water to bathtub / 1日当たりの浴槽水栓さし湯におけるバックアップボイラーが分担する給湯熱負荷 ( MJ/d );  
$L_{BB,ba2}$ is the daily hot water heat load shared by backup boiler by rewarming the bathtub / 1日当たりの浴槽追焚におけるバックアップボイラーが分担する給湯熱負荷 ( MJ/d );  
$e_{BB,k}$ is the daily average backup boiler efficiency by the kitchen / 台所水栓における日平均バックアップボイラー効率  
$e_{BB,s}$ is the daily average backup boiler efficiency by the shower / 浴室シャワー水栓におけるバ日平均バックアップボイラー効率  
$e_{BB,w}$ is the daily average backup boiler efficiency by the washroom / 洗面水栓における日平均バックアップボイラー効率  
$e_{BB,b1}$ is the daily average backup boiler efficiency by filling the bathtub manually / 浴槽水栓湯はりにおける日平均バックアップボイラー効率  
$e_{BB,b2}$ is the daily average backup boiler efficiency by filling the bathtub automattically / 浴槽自動湯はりにおける日平均バックアップボイラー効率  
$e_{BB,ba1}$ is the daily average backup boiler efficiency by addition of hot water to bathtub / 浴槽水栓さし湯における日平均バックアップボイラー効率  
$e_{BB,ba2}$ is the daily average backup boiler efficiency by rewarming the bathtub / 浴槽追焚における日平均バックアップボイラー効率.

In [11]:
def get_E_F_BB( L_BB_k, L_BB_s, L_BB_w, L_BB_b1, L_BB_b2, L_BB_ba1, L_BB_ba2, e_BB_k, e_BB_s, e_BB_w, e_BB_b1, e_BB_b2, e_BB_ba1, e_BB_ba2 ):
    return L_BB_k / e_BB_k, L_BB_s / e_BB_s, L_BB_w / e_BB_w, L_BB_b1 / e_BB_b1, L_BB_b2 / e_BB_b2, L_BB_ba1 / e_BB_ba1, L_BB_ba2 / e_BB_ba2

#### 2) Efficiency of Backup Boiler When Hot Water is Used / 給湯使用時のバックアップボイラーの効率

The daily average backup boiler efficiency by the kitchen, by the washroom, by the shower, by filling the bathtub manually, by filling the bathtub automattically, by addition of hot water to bathtub, by rewarming the bathtub ($e_{BB,k}$, $e_{BB,w}$, $e_{BB,s}$, $e_{BB,b1}$, $e_{BB,b2}$, $e_{BB,ba1}$ and $e_{BB,ba2}$) are depend on the daily hot water heat load $L_{BB,k}$, $L_{BB,w}$, $L_{BB,s}$, $L_{BB,b1}$, $L_{BB,b2}$, $L_{BB,ba1}$, $L_{BB,ba2}$ shared by backup boiler, and the average outside air temperature $\theta_{ex,ave}$ and the efficiency of backup boiler water heater.   
The calculation method shall be in accordance with Appendix C and Appendix D of Chapter 7, Section 1.  
In that case, we replace the hot water supply heat load $L_{BB}$ with the hot water supply heat load $L''$ corrected for solar heat.  
The heat source type of the backup boiler and the efficiency of the water heater are in accordance with Appendix A.

In [9]:
def get_e_BB( BBFuelType, theta_ex_ave, e_rtd, L_2dash_k, L_2dash_s, L_2dash_w, L_2dash_b1, L_2dash_b2, L_2dash_ba1, L_2dash_ba2 ):
    if BBFuelType == 'gas':
        f_hs = ( 0.8754 * e_rtd + 0.060 ) / 0.745
        a_k,   b_k,   c_k   = 0.0019 * f_hs,  0.0013 * f_hs, 0.6533 * f_hs
        a_s,   b_s,   c_s   = 0.0006 * f_hs,  0.0005 * f_hs, 0.7414 * f_hs
        a_w,   b_w,   c_w   = 0.0019 * f_hs,  0.0013 * f_hs, 0.6533 * f_hs
        a_b1,  b_b1,  c_b1  = 0.0000 * f_hs,  0.0002 * f_hs, 0.7839 * f_hs
        a_b2,  b_b2,  c_b2  = 0.0000 * f_hs, -0.0005 * f_hs, 0.7828 * f_hs
        a_ba1, b_ba1, c_ba1 = 0.0000 * f_hs,  0.0002 * f_hs, 0.7839 * f_hs
        a_ba2, b_ba2, c_ba2 = 0.0033 * f_hs,  0.0194 * f_hs, 0.5776 * f_hs
        e_k   = min( a_k   * theta_ex_ave + b_k   * ( L_2dash_k + L_2dash_w ) + c_k,   1.0 )
        e_s   = min( a_s   * theta_ex_ave + b_s   *   L_2dash_s               + c_s,   1.0 )
        e_w   = min( a_w   * theta_ex_ave + b_w   * ( L_2dash_k + L_2dash_w ) + c_w,   1.0 )
        e_b1  = min( a_b1  * theta_ex_ave + b_b1  *   L_2dash_b1              + c_b1,  1.0 )
        e_b2  = min( a_b2  * theta_ex_ave + b_b2  *   L_2dash_b2              + c_b2,  1.0 )
        e_ba1 = min( a_ba1 * theta_ex_ave + b_ba1 *   L_2dash_ba1             + c_ba1, 1.0 )
        e_ba2 = min( a_ba2 * theta_ex_ave + b_ba2 *   L_2dash_ba2             + c_ba2, 1.0 )
    elif BBFuelType == 'kerosene':
        f_hs = ( 0.8669 * e_rtd + 0.091 ) / 0.796
        a_k,   b_k,   c_k   = 0.0005 * f_hs,  0.0028 * f_hs, 0.6818 * f_hs
        a_s,   b_s,   c_s   = 0.0024 * f_hs,  0.0021 * f_hs, 0.7560 * f_hs
        a_w,   b_w,   c_w   = 0.0005 * f_hs,  0.0028 * f_hs, 0.6818 * f_hs
        a_b1,  b_b1,  c_b1  = 0.0000 * f_hs, -0.0027 * f_hs, 0.9026 * f_hs
        a_b2,  b_b2,  c_b2  = 0.0000 * f_hs, -0.0024 * f_hs, 0.8885 * f_hs
        a_ba1, b_ba1, c_ba1 = 0.0000 * f_hs, -0.0027 * f_hs, 0.9026 * f_hs
        a_ba2, b_ba2, c_ba2 = 0.0062 * f_hs,  0.0462 * f_hs, 0.4001 * f_hs
        e_k   = min( a_k   * theta_ex_ave + b_k   * ( L_2dash_k + L_2dash_w ) + c_k,   1.0 )
        e_s   = min( a_s   * theta_ex_ave + b_s   *   L_2dash_s               + c_s,   1.0 )
        e_w   = min( a_w   * theta_ex_ave + b_w   * ( L_2dash_k + L_2dash_w ) + c_w,   1.0 )
        e_b1  = min( a_b1  * theta_ex_ave + b_b1  *   L_2dash_b1              + c_b1,  1.0 )
        e_b2  = min( a_b2  * theta_ex_ave + b_b2  *   L_2dash_b2              + c_b2,  1.0 )
        e_ba1 = min( a_ba1 * theta_ex_ave + b_ba1 *   L_2dash_ba1             + c_ba1, 1.0 )
        e_ba2 = min( a_ba2 * theta_ex_ave + b_ba2 *   L_2dash_ba2             + c_ba2, 1.0 )
    else:
        raise ValueError('バックアップボイラーの燃料種別に指定された値が不正です。')
    return e_k, e_s, e_w, e_b1, e_b2, e_ba1, e_ba2

#### 3) Hot Water Heat Load Shared by Backup Boiler / バックアップボイラーが分担する給湯熱負荷

$$
L_{BB,k} = L_{BB,DHW} \times \frac{L''_{k}}{L_{DHW}}
$$

$$
L_{BB,s} = L_{BB,DHW} \times \frac{L''_{s}}{L_{DHW}}
$$

$$
L_{BB,w} = L_{BB,DHW} \times \frac{L''_{w}}{L_{DHW}}
$$

$$
L_{BB,b1} = L_{BB,DHW} \times \frac{L''_{b1}}{L_{DHW}}
$$

$$
L_{BB,b2} = L_{BB,DHW} \times \frac{L''_{b2}}{L_{DHW}}
$$

$$
L_{BB,ba1} = L_{BB,DHW} \times \frac{L''_{ba1}}{L_{DHW}}
$$

Where  
$L_{BB,DHW}$ is the daily hot water supply heat load shared by backup boiler / 1日当たりの給湯のバックアップボイラーが分担する給湯熱負荷 ( MJ/d );  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷 ( MJ/d );  
$L''_{k}$ is the daily hot water supply hea load corrected for solar heat by the kitchen / 1日当たりの台所水栓における太陽熱補正給湯熱負荷 ( MJ/d );  
$L''_{w}$ is the daily hot water supply hea load corrected for solar heat by the washroom / 1日当たりの洗面水栓における太陽熱補正給湯熱負荷 ( MJ/d );  
$L''_{s}$ is the daily hot water supply hea load corrected for solar heat by the shower / 1日当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷 ( MJ/d );  
$L''_{b1}$ is the daily hot water supply hea load corrected for solar heat by filling the bathtub manually / 1日当たりの浴槽水栓湯はりにおける太陽熱補正給湯熱負荷 ( MJ/d );  
$L''_{b2}$ is the daily hot water supply hea load corrected for solar heat by filling the bathtub automattically / 1日当たりの浴槽自動湯はりにおける太陽熱補正給湯熱負荷 ( MJ/d );  
$L''_{ba1}$ is the daily hot water supply hea load corrected for solar heat by addition of hot water to bathtub / 1日当たりの浴槽水栓さし湯における太陽熱補正給湯熱負荷 ( MJ/d ).

In [15]:
def get_L_BB( L_BB_DHW, L_DHW, L2dash_k, L2dash_s, L2dash_w, L2dash_b1, L2dash_b2, L2dash_ba1 ):
    return L_BB_DHW * L_2dash_k / L_DHW, L_BB_DHW * L_2dash_s / L_DHW, L_BB_DHW * L_2dash_w / L_DHW, \
           L_BB_DHW * L_2dash_b1 / L_DHW, L_BB_DHW * L_2dash_b2 / L_DHW, L_BB_DHW * L_2dash_ba1 / L_DHW 

$$
L_{BB,DHW} = L_{DHW} - Q_{gen,DHW}
$$

Where  
$Q_{gen,DHW}$ is the daily exhaust heat utilization of hot water supply / 1日当たりの給湯の排熱利用量 ( MJ/d ).  

In [17]:
def get_L_BB_DHW( L_DHW, Q_gen_DHW ):
    return L_DHW - Q_gen_DHW

### 3.4 Fuel Consumption of Backup Boiler and Power Consumption of Auxiliary Machine during Hot Water Heating in case of using Exhaust Heat to Hot Water Heating / 温水暖房への排熱利用がある場合の温水暖房時のバックアップボイラーの燃料消費量及び補機の消費電力

#### 1) Power Consumption of Auxiliaries / 補機消費電力量

The auxiliary machine power consumption $E_{E,BB,aux.HWH}$ of backup boiler during hot water heating is set to 0.

In [19]:
def get_E_E_BB_aux_HWH_EHforHWH():
    return 0.0

#### 2) Fuel Consumption / 燃料消費量

$$
E_{F,BB,HWH} = \sum_{t} E_{F,BB,HWH,t}
$$

Where  
$E_{F_BB_HWH}$ is the daily fuel consumption of backup boiler during hot water heating ( MJ/d )  
$E_{F,BB,HWH,t}$ is the hourly fuel consumption of backup boiler during hot water heating at $t$ o'clock / 時刻$t$における1時間当たりの温水暖房時のバックアップボイラーの燃料消費量 ( MJ/h ).  

In [21]:
def get_E_F_BB_HWH_EHforHWH( E_F_BB_HWH_t ):
    return sum( E_F_BB_HWH_t )

The hourly fuel consumption of the backup boiler during hot water heating depends on the type of backup boiler $E_{F,BB,HWH,t}$, the rated  efficiency of the backup boiler of hot water heating, the rated capacity of the backup boiler of hot water heating, the hourly output $Q_{out,BB,HWH}$ of the backup boiler during hot water heating and the hot water temperature $\theta
_{sw,HWH}$ of hot water heating.   
If the backup boiler is a petroleum heat source, it shall be according to Appendix A of Chapter 4 Section 7.   
If the backup boiler is a gas heat source, it shall be according to Appendix B of Chapter 4 Section 7.   
The type of backup boiler of hot water heating, the rated efficiency and the rated capacity are accordance with Appendix A.

In [25]:
def get_E_F_BB_HWH_t_EHforHWH( BBFuelType, BBHEType, e_rtd, q_rtd_hs, Q_out_H_hs, theta_SW_hs, theta_RW_hs ):
    if BBFuelType == 'gas':
        if BBHEType == 'latent':
            if theta_SW_hs == 60.0:
                Q_body = 225.26 * 3600 * 10**(-6)
                f_rtd = 1.038
            elif theta_SW_hs == 40.0:
                Q_body = 123.74 * 3600 * 10**(-6)
                f_rtd = 1.064
            else:
                raise ValueError('送水温度が不正です。')
        else:
            Q_body = 240.96 * 3600 * 10**(-6)
            f_rtd = 0.985
        e_ex = e_rtd * f_rtd * ( q_rtd_hs * 3600 * 10**(-6) + Q_body ) / ( q_rtd_hs * 3600 * 10**(-6) )
        E_G_hs = ( Q_out_H_hs + Q_body ) / e_ex if Q_out_H_hs > 0.0 else 0.0
        return E_G_hs
    elif BBFuelType == 'kerosene':
        theta_SW_hs_rtd = 70.0
        theta_RW_hs_rtd = 50.0
        if theta_RW_hs_rtd >= 46.5:
            if theta_RW_hs >= 46.5:
                f_rtd = 0.9768
            else:
                f_rtd = ( -0.0023 * theta_RW_hs + 1.014 ) / 0.907 * 0.9768
        else:
            if theta_RW_hs >= 46.5:
                f_rtd = 0.907 / ( -0.0023 * theta_RW_hs_rtd + 1.014 ) * 0.9768
            else:
                f_rtd = ( -0.0023 * theta_RW_hs + 1.014 ) / ( -0.0023 * theta_RW_hs_rtd + 1.014 ) * 0.9768
        Q_body_rtd = ( 5.3928 * theta_SW_hs_rtd - 71.903 ) * 3600 * 10**(-6) if BBHEType == 'latent' else 234.0 * 3600 * 10**(-6)
        e_ex = e_rtd * f_rtd * ( q_rtd_hs * 3600 * 10**(-6) + Q_body_rtd ) / ( q_rtd_hs * 3600 * 10**(-6) )
        Q_body = ( 5.3928 * theta_SW_hs - 71.903 ) * 3600 * 10**(-6) if BBHEType == 'latent' else 234.0 * 3600 * 10**(-6)
        E_k_hs = ( Q_out_H_hs + Q_body ) / e_ex if Q_out_H_hs > 0.0 else 0.0
        return E_k_hs
    else:
        raise ValueError('バックアップボイラーの燃料種別に指定された値が不正です。')

$$
Q_{out,BB,HWH,t} = min(L_{BB,HWH,t}, Q_{max,BB,HWH})
$$

Where  
$L_{BB,HWH,t}$ is the hourly heat load of hot water heating shared by backup boiler during hot water heating / 1時間当たりの温水暖房時のバックアップボイラーが分担する温水暖房の熱負荷 ( MJ/h );  
$Q_{max,BB,HWH}$ is the hourly maximum output of backup boiler during hot water heating / 1時間当たりの温水暖房時のバックアップボイラーの最大出力 ( MJ/h ).  

In [27]:
def get_Q_out_BB_HWH_t_EHforHWH( L_BB_HWH_t, Q_max_BB_HWH ):
    return min( L_BB_HWH_t, Q_max_BB_HWH )

The hourly maximum output $Q_{max,BB,HWH}$ of the backup boiler during hot water heating depends on the rated capacity of the backup boiler during hot water heating. If the backup boiler is a petroleum heat source, it shall be according to Appendix A of Chapter 4  Section 7. If the backup boiler is a gas heat source, it shall be according to Appendix B of Chapter 4  Section 7.

In [26]:
def getDailyEnergyConsumptionAndGeneration( spec, day, hour ):
    
    if exhaust_heat_used_for_HWH:
        Q_out_BB_HWH_t = get_Q_out_BB_HWH_t_EHforHWH( L_BB_HWH_t, Q_max_BB_HWH )
        E_F_BB_HWH_t = get_E_F_BB_HWH_t_EHforHWH( BBFuelType, BBHEType, e_rtd, q_rtd_hs, Q_out_BB_HWH_t, theta_SW_hs, theta_RW_hs )
        E_F_BB_HWH = get_E_F_BB_HWH_EHforHWH( E_F_BB_HWH_t )
        E_E_BB_aux_HWH = get_E_E_BB_aux_HWH_EHforHWH()
        L_BB_DHW = get_L_BB_DHW( L_DHW, Q_gen_DHW )
        L_BB_k, L_BB_s, L_BB_w, L_BB_b1, L_BB_b2, L_BB_ba1 = get_L_BB( L_BB_DHW, L_DHW, L2dash_k, L2dash_s, L2dash_w, L2dash_b1, L2dash_b2, L2dash_ba1 )
        e_BB_k, e_BB_s, e_BB_w, e_BB_b1, e_BB_b2, e_BB_ba1, e_BB_ba2 = get_e_BB( BBFuelType, theta_ex_ave, e_rtd, L_2dash_k, L_2dash_s, L_2dash_w, L_2dash_b1, L_2dash_b2, L_2dash_ba1, L_2dash_ba2 )
        E_F_BB_k, E_F_BB_s, E_F_BB_w, E_F_BB_b1, E_F_BB_b2, E_F_BB_ba1, E_F_BB_ba2 = get_E_F_BB( L_BB_k, L_BB_s, L_BB_w, L_BB_b1, L_BB_b2, L_BB_ba1, L_BB_ba2, e_BB_k, e_BB_s, e_BB_w, e_BB_b1, e_BB_b2, e_BB_ba1, e_BB_ba2 )
        E_F_BB_DHW = get_E_F_BB_DHW( E_F_BB_k, E_F_BB_s, E_F_BB_w, E_F_BB_b1, E_F_BB_b2, E_F_BB_ba1, E_F_BB_ba2 )
        E_E_CG_gen = E_E_CG_gen_EHforHWH( E_E_gen_PU, E_E_BB_aux_ba2, E_E_BB_aux_HWH, E_E_TU_aux )
        E_CG = get_E_CG( E_F_PU, E_F_BB_DHW, E_F_BB_HWH )
    else:
        L_BB_DHW = get_L_BB_DHW( L_DHW, Q_gen_DHW )
        L_BB_k, L_BB_s, L_BB_w, L_BB_b1, L_BB_b2, L_BB_ba1 = get_L_BB( L_BB_DHW, L_DHW, L2dash_k, L2dash_s, L2dash_w, L2dash_b1, L2dash_b2, L2dash_ba1 )
        e_BB_k, e_BB_s, e_BB_w, e_BB_b1, e_BB_b2, e_BB_ba1, e_BB_ba2 = get_e_BB( BBFuelType, theta_ex_ave, e_rtd, L_2dash_k, L_2dash_s, L_2dash_w, L_2dash_b1, L_2dash_b2, L_2dash_ba1, L_2dash_ba2 )
        E_F_BB_k, E_F_BB_s, E_F_BB_w, E_F_BB_b1, E_F_BB_b2, E_F_BB_ba1, E_F_BB_ba2 = get_E_F_BB( L_BB_k, L_BB_s, L_BB_w, L_BB_b1, L_BB_b2, L_BB_ba1, L_BB_ba2, e_BB_k, e_BB_s, e_BB_w, e_BB_b1, e_BB_b2, e_BB_ba1, e_BB_ba2 )
        E_F_BB_DHW = get_E_F_BB_DHW( E_F_BB_k, E_F_BB_s, E_F_BB_w, E_F_BB_b1, E_F_BB_b2, E_F_BB_ba1, E_F_BB_ba2 )
        E_E_CG_gen = E_E_CG_gen( E_E_gen_PU, E_E_BB_aux_ba2, E_E_TU_aux )
        E_CG = get_E_CG( E_F_PU, E_F_BB_DHW, E_F_BB_HWH )
        
    return E_CG, E_E_CG_gen

The hourly heat load $L_{BB,HWH,t}$ of hot water heating shared by backup boiler during hot water heating at $t$ o'clock is calculated by the equation below.

$$\large
L_{BB,HWH,t} = \left\{ \begin{array}{ll}
    L_{BB,HWH} \times \frac{L_{HWH,t}}{L_{HWH}}&(L_{HWH} \neq 0)\\ 
    0&(L_{HWH} = 0)\\ 
    \end{array}
\right.
\tag{9}
$$

Where  
$L_{BB,HWH}$ is the daily heat load of hot water heating shared by backup boiler during hot water heating. / 1時間当たりの温水暖房時のバックアップボイラーが分担する温水暖房の熱負荷(MJ/day)    
$L_{HWH,t}$ is the hourly heat load of hot water heating at $t$ o'clock. / 時刻$t$における1時間当たりの温水暖房の熱負荷(MJ/h)    
$L_{HWH}$ is the daily heat load of hot water heating. / 1日当たりの温水暖房の熱負荷(MJ/day)    

The daily heat load $L_{BB,HWH}$ of hot water heating shared by backup boiler during hot water heating is calculated by the equation below.

$$\large
L_{BB,HWH} = L_{HWH} - Q_{gen,HWH}
\tag{10}
$$

Where  
$Q_{gen,HWH}$ is the daily exhaust heat consumption of hot water heating. / 1日当たりの温水暖房の排熱利用量(MJ/day) 

## Power Generation Unit / 発電ユニット

### 1. Exhaust Heat Utilization / 排熱利用量

The daily exhaust heat utilization $Q_{gen,DHW}$ of hot water supply and the daily exhaust heat utilization $Q_{gen,HWH}$ of hot water heating are calculated by the equations below, depending on whether exhaust heat is used for hot water heating and how exhaust heat is used.   
Whether exhaust heat is used for hot water heating and how exhaust heat is used are in accordance with Appendix A.

&#9312;When there is no use of exhaust heat to hot water heating:<br><br>
$$\large
Q_{gen,DHW} = min(Q_{PU,gen} \times r_{DHW,gen,PU},L_{DHW})
\tag{11a}
$$

$$\large
Q_{gen,HWH} = 0.0
\tag{11b}
$$
<br>

&#9313;In case of using exhaust heat to hot water heating, models with hot water supply priority: <br><br>
$$\large
Q_{gen,DHW} = min(Q_{PU,gen} \times r_{DHW,gen,PU},L_{DHW})
\tag{12a}
$$

$$\large
Q_{gen,HWH} = min((Q_{PU,gen} - Q_{gen,DHW}) \times r_{HWH,gen,PU},L_{HWH})
\tag{12b}
$$
<br>

&#9314;In case of using exhaust heat to hot water heating, models with hot water heating priority: <br><br>
$$\large
Q_{gen,HWH} = min(Q_{PU,gen} \times r_{HWH,gen,PU},L_{HWH})
\tag{13a}
$$

$$\large
Q_{gen,DHW} = min((Q_{PU,gen} - Q_{gen,HWH}) \times r_{DHW,gen,PU},L_{DHW})
\tag{13b}
$$
<br>


Where  
$Q_{PU,gen}$ is the daily exhaust heat quantity of power generation unit. / 1日当たりの発電ユニット排熱量(MJ/day)  
$r_{DHW,gen,PU}$ is the exhaust heat utilization rate at hot water supply in power generation unit. / 発電ユニットの給湯排熱利用率  
$r_{HWH,gen,PU}$ is the exhaust heat utilization rate at hot water heating in power generation unit. / 発電ユニットの温水暖房排熱利用率  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub. / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/day)  

The exhaust heat utilization rate $r_{DHW,gen,PU}$ at hot water supply and the exhaust heat utilization rate $r_{HWH,gen,PU}$ at hot water heating in power generation unit are in accordance with Appendix A.

The daily exhaust heat quantity $Q_{PU,gen}$ of power generation unit is calculated by the equation below.

$$\large
Q_{PU,gen} = E_{F,PU} \times e_{H,PU}
\tag{14}
$$

Where  
$E_{F,PU}$ is the daily fuel consumption of power generation unit. / 1日当たりの発電ユニットの燃料消費量(MJ/day)  
$e_{H,PU}$ is the daily average exhaust heat efficiency of power generation unit. / 発電ユニットの日平均排熱効率  

### 2. Electric-generating Capacity / 発電量

The hourly power generation $E_{E,gen,PU,t}$ of power generation unit at $t$ o'clock is calculated by the equation below.

$$\large
E_{E,gen,PU,t} = E_{E,gen,PU} \times \frac{E_{E,PU,t}}{E_{E,PU}}
\tag{15}
$$

Where  
$E_{E,gen,PU}$ is the daily power generation of power generation unit. / 1日当たりの発電ユニットの発電量(kWh/day)  
$E_{E,PU,t}$ is the hourly sharing power load of power generation unit. / 1時間当たりの発電ユニットの分担可能電力負荷(kWh/h)  
$E_{E,PU}$ is the daily power load that can be shared by power generation unit. / 1日当たりの発電ユニットの分担可能電力負荷(kWh/day)  

The power generation $E_{E,gen,PU}$ of power generation unit is calculated by the equation below.

$$\large
E_{E,gen,PU} = E_{F,PU} \times e_{E,PU} \div 3.6
\tag{16}
$$

Where  
$E_{F,PU}$ is the daily fuel consumption of power generation unit. / 1日当たりの発電ユニットの燃料消費量(MJ/day)  
$e_{E,PU}$ is the daily average power generation efficiency of power generation unit. / 発電ユニットの日平均発電効率  

### 3. Fuel Consumption / 燃料消費量

The daily fuel consumption $E_{F,PU}$ of power generation unit is calculated by the equations below.  
The power generation method of the power generation unit is accordance with Appendix A.

&#9312; When the power generation method of the power generation unit is "heat main":  

$$\large
E_{F,PU} = min(E_{F,PU,EVt}, E_{F,PU,HVt})
\tag{17a}
$$

&#9313; When the power generation method of the power generation unit is "electricity main":  

$$\large
E_{F,PU} = E_{F,PU,EVt}
\tag{17b}
$$

Where  
$E_{F,PU,EVt}$ is the virtual daily fuel consumption at estimation of power generation amount by power generation unit. / 1日当たりの発電ユニットの発電量推定時の仮想燃料消費量(MJ/day)  
$E_{F,PU,HVt}$ is the virtual daily fuel consumption at estimation of exhaust heat quantity by power generation unit. / 1日当たりの発電ユニットの排熱量推定時の仮想燃料消費量(MJ/day)  

The virtual daily fuel consumption $E_{F,PU,EVt}$ at estimation of power generation amount by power generation unit is calculated by the equation below.

$$\large
E_{F,PU,EVt} = E_{E,gen,PU,EVt} \times 3.6 \div e_{E,PU} 
\tag{18}
$$

Where  
$E_{E,gen,PU,EVt}$ is the virtual daily generation amount at estimation of power generation amount by power generation unit. / 1日当たりの発電ユニットの発電量推定時の仮想発電量(kWh/day)  
$e_{E,PU}$ is the daily average power generation efficiency of power generation unit. / 発電ユニットの日平均発電効率  

The virtual daily generation amount $E_{E,gen,PU,EVt}$ at estimation of the power generation amount of the power generation unit is calculated by the equation below.

$$\large
E_{E,gen,PU,EVt} = min(a_{PU} \times E_{E,PU} \times 3.6 + a_{DHW} \times L_{DHW} + a_{HWH} \times L_{HWH}+ b,\\
\large
E_{E,PU} \times c \times 3.6) \div 3.6
\tag{19}
$$

Where  
$E_{E,PU}$ is the daily power load that can be shared by power generation unit. / 1日当たりの発電ユニットの分担可能電力負荷(kWh/day)  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub. / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/day)  
$L_{HWH}$ is the daily heat load of hot water heating. / 1日当たりの温水暖房の熱負荷(MJ/day)  

The coefficients $a_{PU}$, $a_{DHW}$, $a_{HWH}$, $b$ and $c$ are accordance with Appendix A.

The virtual daily fuel consumption $E_{F,PU,HVt}$ at estimation of exhaust heat quantity by power generation unit is calculated by the equation below only for models using the equation (17a).

$$\large
E_{F,PU,HVt} = (a_{DHW} \times L_{DHW} + a_{HWH} \times L_{HWH}) \times r_{H,gen,PU,HVt} \div e_{H,PU}
\tag{20}
$$

Where  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub. / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/day)  
$L_{HWH}$ is the daily heat load of hot water heating. / 1日当たりの温水暖房の熱負荷(MJ/day)  
$r_{H,gen,PU,HVt}$ is the virtual exhaust heat quantity upper limit ratio at estimation of exhaust heat quantity by power generation unit. / 発電ユニットの排熱量推定時の仮想排熱量上限比  
$e_{H,PU}$ is the daily average exhaust heat efficiency of power generation unit. / 発電ユニットの日平均排熱効率  

The coefficients $a_{DHW}$ and $a_{HWH}$ are accordance with Appendix A.

The virtual exhaust heat quantity upper limit ratio $r_{H,gen,PU,HVt}$ at estimation of exhaust heat quantity by power generation unit is calculated by the equation below.

$$\large
r_{H,gen,PU,HVt} = a_{DHW} \times L_{DHW} + a_{HWH} \times L_{HWH} + b
\tag{21}
$$

Where  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub. / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/day)  
$L_{HWH}$ is the daily heat load of hot water heating. / 1日当たりの温水暖房の熱負荷(MJ/day)  

The coefficients $a_{DHW}$, $a_{HWH}$ and $b$ are accordance with Appendix A.

### 4. Power Generation Efficiency / 発電効率

The daily average power generation efficiency $e_{E,PU}$ of power generation unit is calculated by the equation below.  
However, the value obtained by the following equation is assumed to be equal to the upper limit value when it exceeds the upper limit value of the power generation efficiency of the power generation unit and equal to the lower limit value if it is lower than the lower limit value.

$$\large
e_{E,PU} = a_{PU} \times E_{E,PU} \times 3.6 + a_{DHW} \times L_{DHW} + a_{HWH} \times L_{HWH} + b
\tag{22}
$$

Where  
$E_{E,PU}$ is the daily power load that can be shared by power generation unit. / 1日当たりの発電ユニットの分担可能電力負荷(kWh/day)  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub. / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/day)  
$L_{HWH}$ is the daily heat load of hot water heating. / 1日当たりの温水暖房の熱負荷(MJ/day)  

The coefficients $a_{PU}$, $a_{DHW}$, $a_{HWH}$, $b$, the upper limit value and the lower limit value are accordance with Appendix A.

### 5. Exhaust Heat Efficiency / 排熱効率

The daily average exhaust heat efficiency $e_{H,PU}$ of power generation unit is calculated by the equation below.  
However, the value obtained by the following equation is assumed to be equal to the upper limit value when it exceeds the upper limit value of the exhaust heat efficiency of the power generation unit and equal to the lower limit value if it is lower than the lower limit value.

$$\large
e_{H,PU} = a_{PU} \times E_{E,PU} \times 3.6 + a_{DHW} \times L_{DHW} + a_{HWH} \times L_{HWH} + b
\tag{23}
$$

Where  
$E_{E,PU}$ is the daily power load that can be shared by power generation unit. / 1日当たりの発電ユニットの分担可能電力負荷(kWh/day)  
$L_{DHW}$ is the daily hot water supply heat load corrected for solar heat except rewarming the bathtub. / 1日当たりの浴槽追焚を除く太陽熱補正給湯熱負荷(MJ/day)  
$L_{HWH}$ is the daily heat load of hot water heating. / 1日当たりの温水暖房の熱負荷(MJ/day)  

The coefficients $a_{PU}$, $a_{DHW}$, $a_{HWH}$, $b$, the upper limit value and the lower limit value are accordance with Appendix A.

### 6. Shareable Electric Load / 分担可能電力負荷

The daily power load $E_{E,PU}$ that can be shared by power generation unit is calculated by the equation below.

$$\large
E_{E,PU} = \sum E_{E,PU,t}
\tag{24}
$$

Where  
$E_{E,PU,t}$ is the hourly power load that can be shared by power generation unit at $t$ o'clock. / 時刻$t$における1時間当たりの発電ユニットの分担可能電力負荷(kWh/h)  

The hourly power load $E_{E,PU,t}$ that can be shared by power generation unit at $t$ o'clock is calculated by the equation below.

$$\large
E_{E,PU,t} = min(E_{E,dmd,PU,t}, P_{rtd,PU} \times 10^{-3})
\tag{25}
$$

Where  
$E_{E,dmd,PU,t}$ is the hourly power demand of power generation unit at $t$ o'clock. / 時刻$t$における1時間当たりの発電ユニットの電力需要(kWh/h)  
$P_{rtd,PU}$ is the rated power generation output. / 定格発電出力(W)  

The rated power generation output $P_{rtd,PU}$ is accordance with Appendix A.

The power demand $E_{E,dmd,PU}$ of power generation unit is calculated by the equations below.

&#9312;When there is no use of exhaust heat to hot water heating:<br><br>
$$\large
E_{E,dmd,PU} = E_{E,dmd} + E_{E,BB,aux,ba2} + E_{E,BB,aux,HWH} + E_{E,TU,aux}
\tag{26a}
$$

<br>
&#9313;When exhaust heat is used for hot water heating: <br><br>
$$\large
E_{E,dmd,PU} = E_{E,dmd} + E_{E,BB,aux,ba2} + E_{E,TU,aux}
\tag{26b}
$$

Where  
$E_{E,dmd}$ is the hourly power demand. / 1時間当たりの電力需要(kWh/h)  
$E_{E,BB,aux,ba2}$ is the hourly auxiliary machine power consumption of backup boiler during rewarming the bathtub. / 1時間当たりの浴槽追焚時のバックアップボイラーの補機消費電力量(kWh/h)  
$E_{E,BB,aux,HWH}$ is the hourly auxiliary machine power consumption of backup boiler during hot water heating. / 1時間当たりの温水暖房時のバックアップボイラーの補機消費電力量(kWh/h)  
$E_{E,TU,aux}$ is the hourly auxiliary machine power consumption of the tank unit. / 1時間当たりのタンクユニットの補機消費電力量(kWh/h)  

## Fuel Consumption of Backup Boiler and Power Consumption of Auxiliary Machine during Hot Water Heating When There Is No Use of Exhaust Heat to Hot Water Heating / 温水暖房への排熱利用がない場合の温水暖房時のバックアップボイラーの燃料消費量と補機消費電力

### 1. Power Consumption of Auxiliaries / 補機消費電力

The hourly auxiliary machine power consumption $E_{E,BB,aux,HWH}$ of backup boiler during hot water heating occures when the hourly output $Q_{out,BB,aux,HWH}$ of backup boiler of hot water heating is greater than 0.  
The calculation method is as follows:  
If the backup boiler is a petroleum heat source, it shall be according to Appendix A in Chapter 4 Section 7.  
If the backup boiler is a gas heat source, it shall be according to Appendix B in Chapter 4 Section 7.  

In that case, we replace 
the power consumption $E_{E,hs}$ in the appendix with the hourly auxiliary machine power consumption $E_{E,BB,aux,HWH}$ of backup boiler during hot water heating, 
the hot water supply operation ratee $r_{WS,hs}$ of hot water heating source machin with the hot water supply operation rate $r_{WS,HWH}$ of hot water heating, 
and 
the kerosene consumption $E_{K,hs}$ of heat source equipment for hot water heating in Appendix A or the gas consumption $E_{G,hs}$ of the heat source equipment for hot water heating in Appendix B with the fuel consumption $E_{F,BB,HWH}$ of hot water heating backup boiler.

### 2. Fuel Consumption / 燃料消費量

The daily fuel consumption $E_{F,BB,HWH}$ of backup boiler during hot water heating is calculated by the equation below.  This is a value obtained by integrating the hourly fuel consumption $E_{F,BB,HWH,t}$ of the backup boiler of hot water heating at $t$ o'clock every day.

$$\large
E_{F,BB,HWH} = \sum E_{F,BB,HWH,t}
\tag{27}
$$

Where  
$E_{F,BB,HWH,t}$ is the hourly fuel consumption of backup boiler during hot water heating at $t$ o'clock. / 時刻$t$における1時間当たりの温水暖房時のバックアップボイラーの燃料消費量(MJ/h)  

The hourly fuel consumption $E_{F,BB,HWH}$ of the backup boiler during hot water heating depends on the type of backup boiler, the rating efficiency of the backup boiler of hot water heating, the rated capacity of the backup boiler of hot water heating, the hourly output $Q_{out,BB,HWH}$ of the backup boiler during hot water heating and the hot water temperature $\theta_{sw,HWH}$.  
If the backup boiler is a petroleum heat source, it shall be according to Appendix A of Chapter 4 Section 7.   
If the backup boiler is a gas heat source, it shall be according to Appendix B of Chapter 4 Section 7.   
The type of hot water heating backup boiler, the rated efficiency and the rated capacity are accordance with Appendix A.  

The hourly output $Q_{out,BB,HWH}$ of backup boiler of hot water heating is calculated by the equation below.

$$\large
Q_{out,BB,HWH} = min(L_{BB,HWH}, Q_{max,BB,HWH})
\tag{28}
$$

Where  
$L_{BB,HWH}$ is the hourly heat load of hot water heating shared by backup boiler during hot water heating. / 1時間当たりの温水暖房時のバックアップボイラーが分担する温水暖房の熱負荷(MJ/h)  
$Q_{max,BB,HWH}$ is the hourly maximum output of backup boiler during hot water heating. / 1時間当たりの温水暖房時のバックアップボイラーの最大出力(MJ/h)  

The hourly maximum output $Q_{max,BB,HWH}$ of the backup boiler during hot water heating depends on the rated capacity of the backup boiler during hot water heating.   
If the backup boiler is a petroleum heat source, it shall be according to Appendix A of Chapter 4  Section 7.   
If the backup boiler is a gas heat source, it shall be according to Appendix B of Chapter 4  Section 7.

The hourly heat load $L_{BB,HWH}$ of hot water heating shared by backup boiler is calculated by the equation below.

$$\large
L_{BB,HWH} = L_{HWH}
\tag{29}
$$

## Rewarming the Bathtub / 浴槽追焚

The daily power consumption $E_{E,BB,aux,ba2}$ of auxiliary machine by rewarming the bathtub occures when the daily hot water heat load $L_{BB,ba2}$ shared by the backup boiler by rewarming the bathtub is greater than 0.   
The calculation method is the daily power consumption $E_{E,hs,aux3}$ by auxiliaries when keeping hot water heaters in Appendix C or Appendix D of Chapter 7 Section 1.   
In that case, we replace the hot water supply heat load $ L''_{ba2}$ corrected for solar heat with $L_{BB,ba2}$.  

Rewarming the bathtub shall occur at 19 o'clock.

$$\large
E_{E,BB,aux,ba2,t} = \left\{ \begin{array}{ll}
    E_{E,BB,aux,ba2}&(t = 19)\\ 
    0&(t \neq 19)\\ 
    \end{array}
\right.
\tag{30}
$$

For hot water supply heat load corrected for solar heat in rewarming the bathtub, exhaust heat is sometimes used depending on the model, but its usage is small compared with exhaust heat usage to hot water supply heat load corrected for solar heat for other uses, it can be considered that exhaust heat utilization is not performed. 
In other words, the hot water supply heat load $L_{BB,ba2}$ shared by the backup boiler in rewarming the bathtub is supposed to be equal to the solar heat correction hot water supply load in rewarming the bathtub, which is calculated by the equation below.

$$\large
L_{BB,ba2} = L''_{ba2}
\tag{31}
$$

Where  
$L''_{ba2}$ is the daily hot water supply heat load corrected for solar heat by rewarming the bathtub. / 1日当たりの浴槽追焚の太陽熱補正給湯熱負荷(MJ/day)  

## Power Consumption of Auxiliaries of Tank Unit / タンクユニットの補機消費電力

The hourly power consumption $E_{E,TU,aux}$ of auxiliaries of tank unit is calculated by the equation below.

&#9312;When there is no use of exhaust heat to hot water heating:<br><br>
$$\large
E_{E,TU,aux} = P_{TU,aux,DHW} \times 10^{-3}
\tag{32a}
$$

<br>
&#9313;When exhaust heat is used for hot water heating: <br><br>
$$\large
E_{E,TU,aux} = (P_{TU,aux,DHW} + P_{TU,aux,HWH} \times r_{WS,HWH}) \times 10^{-3}
\tag{32b}
$$

Where  
$P_{TU,aux,DHW}$ is the auxiliary machine power consumption of the tank unit of hot water supply. / 給湯のタンクユニットの補機消費電力(W)  
$P_{TU,aux,HWH}$ is the auxiliary machine power consumption of the tank unit of hot water heating. / 温水暖房のタンクユニットの補機消費電力(W)  
$r_{WS,HWH}$ is the hot water supply operation rate of hot water heating. / 温水暖房の温水供給運転率  

The power consumption $P_{TU,aux,DHW}$ of auxiliary machine of the tank unit of hot water supply and the power consumption  $P_{TU,aux,HWH}$ of auxiliary machine of the tank unit of hot water heating are accordance with Appendix A.

## Others / その他

The daily hot water supply heat load $L_{DHW}$ except rewarming the bathtub is calculated by the equation below.

$$\large
L_{DHW} = L''_{k} + L''_{w} + L''_{s} + L''_{b1} + L''_{b2} + L''_{ba1}
\tag{33}
$$

Where  
$ L''_{k} $ is the daily hot water supply heat load corrected for solar heat by the kitchen./1日当たりの台所水栓の太陽熱補正給湯熱負荷(MJ/day)  
$ L''_{w} $ is the daily hot water supply heat load corrected for solar heat by the washroom./1日当たりの洗面水栓の太陽熱補正給湯熱負荷(MJ/day)  
$ L''_{s} $ is the daily hot water supply heat load corrected for solar heat by the shower./1日当たりの浴室シャワー水栓の太陽熱補正給湯熱負荷(MJ/day)  
$ L''_{b1} $ is the daily hot water supply heat load corrected for solar heat by filling the bathtub manually./1日当たりの浴槽水栓湯はりの太陽熱補正給湯熱負荷(MJ/day)  
$ L''_{b2} $ is the daily hot water supply heat load corrected for solar heat by filling the bathtub automattically./1日当たりの浴槽自動湯はりの太陽熱補正給湯熱負荷(MJ/day)  
$ L''_{ba1} $ is the daily hot water supply heat load corrected for solar heat by addition of hot water to bathtub./1日当たりの浴槽水栓さし湯の太陽熱補正給湯熱負荷(MJ/day)  

The daily hot water supply heat load $L''_{u}$ corrected for solar heat (where subscript $u$ is a suffix for use ($k$, $s$, $w$, $b1$, $b2$, $ba1$, $ba2$)) shall be the value calculated by Chapter 7 according to the number of residents.  
The number of residents mentioned here is not the actual number of people living in the dwelling unit but the number of resident virtually determined from the total of the floor area of the dwelling unit, and be obtained from Appendix C in Chapter 2.

The daily heat load $L_{HWH}$ of hot water heating is calculated by the equation below.

$$
L_{HWH} = \sum^{23}_{t=0}L_{HWH,t}
\tag{34}
$$

Where  
$L_{HWH,t}$ is the heat load of hot water heating per hour at $t$ o'clock. / ݇時刻$t$における1 時間当たりの温水暖房の熱負荷(MJ/day)

The hourly power demand $E_{E,dmd}$ is calculated by the equation below.

$$\large
E_{E,dmd} = E_{E,H} + E_{E,C} + E_{E,V} + E_{E,L} + E_{E,AP}
\tag{35}
$$

Where  
$E_{E,dmd}$ is the hourly power demand. / 1時間当たりの電力需要(kWh/h)  
$E_{E,H}$ is the hourly power consumption of heating equipment. / 1時間当たりの暖房設備の消費電力量(kWh/h)  
$E_{E,C}$ is the hourly power consumption of cooling equipment. / 1時間当たりの冷房設備の消費電力量(kWh/h)  
$E_{E,V}$ is the hourly power consumption of mechanical ventilation equipment. / 1時間当たりの機械換気設備の消費電力量(kWh/h)  
$E_{E,L}$ is the hourly power consumption of lighting equipment. / 1時間当たりの照明設備の消費電力量(kWh/h)  
$E_{E,AP}$ is the hourly power consumption of home appliances. / 1時間当たりの家電の消費電力量(kWh/h)  

## Integrated Function